In [709]:
%matplotlib inline

# Virtual FCS, using MD Simulation Data

This goal of this notebook is to generate a simulated FCS measurement using data from a GROMACS simulation.

The setup of the virtual system is a membrane with a circularly symmetric incident beam.

Data is read in from an .xtc or .trr file, using a .gro file to define the system topology. Data frames from the input file are iterated through, and for each frame, a detected intensity from each lipid is calculated. The intensity trace and autocorrelation functions for each lipid, and for the total at each frame, are plotted.

## TODOs


#### ☒ Why don't autocorrelation curves look right? Flatness in middle is unexpected
- Done. See below
 
#### ☒ Generate and plot autocorrelation curve prediction. I'm still not understanding something about autocorrelation curves - I don't see where the sigmoid shape comes from.
 - Done. Curves need to be plotted on a semilog scale in order to see the expected sigmoid shape.
 
#### ☒ Try autocorrelating the data myself instead of using acorr. Maybe something weird is happening with the way matplotlib does autocorrelation that isn't desirable.
 - Done. The manually autocorrelated data is identical to calling plt.acorr with normed=True
 
#### ☒ Parallelize data analysis
 - Tried it. The analysis is quite computationally cheap compared to the extra overhead from spawning new threads -- not worth the time saved by running the analysis in parallel.
 
#### ☒ Handle breaking data up into bins better -- currently ignores the remainder of lipids that don't fall into a bin. (I.e. 11 lipids, bin size 3, the remaining 2 that don't evenly fall into bins are discarded.)
 - Done. The leftover lipids are put into their own bin, which may be smaller than the BIN_SIZE.
 
#### ☒ How do PBC vs unwrapped affect prediction of diffusion constant?
 - Done. Don't appear to significant affect results.
 
#### ☐ Curve fit method seems to work better for small spot sizes, but 0.5-crossing method seems better for larger spots? Why?
     
[Checkbox symbols]:<> (☒ ☐)

It's recommended to 'unwrap' simulation data to remove potential artifacts from periodic boundary conditions in the simulation.

### Method 1 (Better for big systems)
The *best* way of doing this requires a trajectory file (i.e. `.xtc`, `.trr`), a `.gro`, and a `.tpr`. By doing this, the input data filesizes can be significantly reduced off the bat by selecting only the lipid groups to keep in the new trajectory file. This can be accomplished by running the following.

When prompted to select a group, select only the group of lipids.

`$>gmx trjconv -f <trajectory file> -s <.gro file> -o <output trajectory file> -pbc nojump`

`$>gmx trjconv -f <.gro file> -s <.tpr file> -o <output .gro file> -pbc nojump`

### Method 2
This can also be done in one step, with only a trajectory file and a `.gro` file by selecting the group of ALL atoms when prompted.

`$>gmx trjconv -f <trajectory file> -s <.gro file> -o <output trajectory file> -pbc nojump`

## Notes on input data

In [710]:
trajectory_file = "40nm/run_nojump.xtc"
trr_file = "40nm/run.trr"
topology_file = "40nm/system.gro"

In [711]:
import mdtraj as md
import numpy as np
from scipy.optimize import curve_fit, fsolve
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

import multiprocessing
from multiprocessing.pool import ThreadPool

import trajectory

plt.rcParams['figure.figsize'] = (20, 12)

## Simulation constants

#### Detection area parameters

`spot_radius` is used to determine whether a particle is within the detection area. Currently, this is unused, as cutoff is determined by the sigma of the beam Gaussian.

In [712]:
# Radius of detection area (in nanometers)
spot_radius = 50 # Currently unused

# Coordinates of detection area center (in nanometers)
spotX = 0.0
spotY = 0.0
spotZ = 10.0 # This isn't used, since we're looking at 2D membranes

#### Gaussian parameters

Note that what's being set here is the std. deviation `w_xy` of the Gaussian - spot size is probably more accurately represented by the FWHM, or `w_xy * 2.3548`

In [713]:
#   Radial and axial std. dev.s of the Gaussian beam profile
FWHM = 50.
w_xy = FWHM / 2.3548
w_z = 2 # Unused
k = w_z/w_xy # Unused, just considering a 2-D membrane

#### Various simulation parameters

`STEP` is the stride used when iterating through data frames. Data frames are taken every timestep.

`INTENSITY` is a scaling constant used to determine the maximum intensity of fluorescence.

`SAMPLING_RATIO` defines the percentage of particles to be 'tagged'. Untagged particles are discarded at the beginning of the simulation.

`CUTOFF` defines a cutoff for the beam profile. This may be useful to avoid artifacts from periodic boundary conditions.

In [740]:
# Step size for iterating through data frames
STEP = 1

# Scaling constant for the intensity of a fluorescing particle
INTENSITY = 1

# Percentage of tagged particles
SAMPLING_RATIO = .1

# How many sigmas out from the beam center to truncate the beam's Gaussian profile at
CUTOFF = 2.5

# How many lipids to bin into a single trajectory
BIN_SIZE = 50

# How many random spots to use
N_SPOTS = 4

#### Diffusion parameters

`D` is the diffusion constant for POPC, ~~using data from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1303347/~~ using values from Andrew's STRD paper$^{[2]}$, in units of nm^2/ns.

`tauD` is the expected diffusion time, or the time of the half-max for the autocorrelation curve.

In [715]:
#D = .01 From NIH paper
D = .0245
D = 4
tauD = FWHM**2 / (4*D)
print("Expected diffusion time is %.2f nanoseconds" % tauD)

## Expected diffusion times for various beam waists

In [716]:
beam_waists = np.arange(10,100,5) # In nanometers
print("beam FWHM (nm)".ljust(16) + "|" + "tau_D (ns)".rjust(15))
print("-"*27)
for v in beam_waists:
    print(str(v).ljust(16) + "|" + str(v ** 2 / (4*D)).rjust(15) )

## Useful functions

### `check_in_detection_volume`
Function to check if a given lipid is within the detection volume.

Right now, we're interested in contributions from all lipids regardless of position, so the `return True` short circuits it. 

In [717]:
def check_in_detection_volume(t, frame_index, residue):

    # For now, pay attention to all atoms, regardless of whether or not they're
    #   in the detection volume. 
    # Keep this function as a placeholder, in case this changes.
    return True

    x, y, z = t.xyz[frame_index, residue._atoms[0].index]

    # Get magnitude of distance to the spot center
    distance = (x - spotX)**2 + (y - spotY)**2

    # Check if the distance is within the spot radius
    in_detection_area = distance <= spot_radius**2

    return in_detection_area

### `generate_detection`
Defines what happens when a detection is generated from a lipid. Right now, `INTENSITY` is weighted by a 2-D Gaussian determined by the cell's position:

$I = I_0 \exp{ \left( - \frac{(x-x_0)^2 + (y-y_0)^2 }{2 \sigma^2} \right) }$

#### Inputs
- `t`: mdtraj.Trajectory object
- `frame_index`: index of frame to analyze in trajectory  
- `atom`: mdtraj.Atom object to be used for detection

#### Returns
- Intensity contribution from `atom`

In [718]:
def generate_detection(t, frame_index, atom, spotcenter):
    
    _x = spotcenter[0]
    _y = spotcenter[1]

    # Get coordinates of residue (more correctly, of the P atom)
    #x, y, z = t.xyz[frame_index, residue._atoms[0].index]
    x, y, z = t.xyz[frame_index, atom.index]
    
    # Get magnitude of distance to the spot center
    distance = (x - _x)**2 + (y - _y)**2
    
    # Truncate at 2 sigma
    if distance > CUTOFF**2 * w_xy**2:
        return 0.0

    # Calculate contribution to intensity from an atom, based on the Gaussian
    #   profile of the incident beam and the particle's position.
    intensity = \
        INTENSITY * np.exp(
        -( distance ) # + ((z - spotZ)/k)**2)
        / (2 * w_xy**2) )

    return intensity

### `analyze_frame`

Analyzes the positions of atoms in a given frame, and updates the `detections` list with each atom's position.

#### Inputs
- `t`: mdtraj.Trajectory object
- `frame_index`: index of frame to analyze in trajectory  
- `detections`: shared list of all detections

In [719]:
def analyze_frame(t, frame_index, detections, spot_number=0, spotcenter=(spotX, spotY)): 
    
    print("\rProcessing frame %d out of %d" % (frame_index/STEP, len(t)/STEP), end="\r")

    # Iterate through each atom remaining in the topology
    for atom in t.topology.atoms:

        # Do analysis if atom is in detection volume
        if not check_in_detection_volume(t, frame_index, atom):
            print("Not in detection volume, skipping.")
            continue

        detected = generate_detection(t, frame_index, atom, spotcenter)
        
        detections[spot_number][atom.index][int(frame_index/STEP)] += detected

### `autocorr_model`

Models the expected autocorrelation curve at a given time, for a given diffusion constant.

#### Inputs
- `t`: Time to calculate autocorrelation at
- `D`: Diffusion constant

#### Returns
- Value of autocorrelation curve at time `t`

In [720]:
def autocorr_model(t, D):
    tauD = FWHM**2 / (4*D)
    
    return (1 + t / tauD)**(-1)

### `autocorrelate`
Computes the autocorrelation curve for a set of data.

In [721]:
def autocorrelate(_data, normed=True):
    if normed:
        normalized = _data/np.linalg.norm(_data)
        
    autocorrelated = np.correlate(normalized, normalized, mode='full')
    autocorrelated = autocorrelated[(autocorrelated.size-1)//2 :]
    
    return autocorrelated

## Load trajectory data

Import FCS data from .xtc file. Also specify a .gro file for the system topology.

**NB:** A .trr file can be used for better resolution, since an .xtc typically uses some compression. However, a .trr is also much larger.

In [722]:
%%time
#t = md.load(trajectory_file, top=topology_file)
#t = md.load_trr(trr_file, top=topology_file)

Calculate timestep for data analysis, given the simulation data timestep and current stride.

In [723]:
# print("Timestep for data analysis is %.2f picoseconds (%.2f nanoseconds)" % (t.timestep * STEP, t.timestep * STEP / 1000))

## Reduce atom selection to only phosphorous atoms

This is a bit of a simplification, but significantly reduces the amount of atoms to iterate over if we're only considering the phosphorous at the center of the phosphate group. Error from this would be on the order of the bond lengths, so roughly 1.5 angstrom.

In [724]:
# print("Starting with %d atoms" % t.topology.n_atoms)

# phosphorous_atoms = [a.index for a in t.topology.atoms if a.element.symbol == 'P']
# t.atom_slice(phosphorous_atoms, inplace=True)

# print("Reduced to %d phosphorous atoms" % t.topology.n_atoms)

# # Reduce to the sampling ratio * number of phosphorous atoms
# num_sampled = int(t.topology.n_atoms * SAMPLING_RATIO)

# # Randomly select the sampled atoms
# sampled = np.random.choice([a.index for a in t.topology.atoms], num_sampled, replace=False)
# t.atom_slice(sampled, inplace=True)

# print("Reduced to %d \"tagged\" phosphorous atoms" % t.topology.n_atoms)

## Import data from Pickle file

In [725]:
import pickle
t = pickle.load(open('../../windrive/linux/output.pkl', 'rb'))
print("Shape is %s, reducing by %d%%." % (np.shape(t.xyz), 100-SAMPLING_RATIO*100))
# t.xyz = t.xyz[:10000]
t.reduce(SAMPLING_RATIO)
print(np.shape(t.xyz))

Create a list of lists to store detected intensity at each timestep for each lipid.

In [726]:
%%time
detections = np.full(shape=(N_SPOTS, t.topology.n_residues, int(np.ceil(len(t)/STEP))), fill_value=0.0)

## Data Analysis

Iterate through each frame of data in the trajectory file, and generate a detected intensity from each lipid (represented by its head group P atom).

In [727]:
spot_centers = np.random.uniform(-750,750, size=(N_SPOTS,2))

def analyze_spot(t, spot, detections, spot_num):
    for frame_index in range(0, len(t), STEP):
        analyze_frame(t, frame_index, detections, spot_num, spot)

pool = ThreadPool()

print("Performing analysis for %d randomly assigned spots, parallelized over %d cores." % (N_SPOTS, multiprocessing.cpu_count()))

for i in range(N_SPOTS):
    print("\nStarting job %d\n" % i)
    pool.apply_async(analyze_spot, args=(t, spot_centers[i], detections, i))
    
pool.close()
pool.join()

### Bin lipids in data

`binned_tots[bin][timestep]` is the total intensity for a certain bin at a certain timestep

`binned_avgs[bin]` is the average intensity for a bin, over the whole time

`binned_dI[bin][timestep]` is the difference of the total intensity from the average at a given timestep

`binned_tot[timestep]` is the total intensity from all bins at a certain timestep

In [741]:
n_bins = int(np.ceil(t.topology.n_residues/BIN_SIZE))

print(t.topology.n_residues)
print(BIN_SIZE)

if not t.topology.n_residues%BIN_SIZE == 0:
    print("Number of residues is not evenly divisible by bin size. Desired size is %d, one bin will contain %d." % (BIN_SIZE, t.topology.n_residues%BIN_SIZE) )

binned_tots = np.ndarray(shape=(N_SPOTS, n_bins, len(t)))
binned_avgs = np.ndarray(shape=(N_SPOTS, n_bins, len(t)))
binned_dI = np.ndarray(shape=(N_SPOTS, n_bins, len(t)))
binned_tot = np.ndarray(shape=(N_SPOTS, len(t)))
    
for spot_num in range(N_SPOTS):
    print("Sorting data into %d groups" % n_bins)

    binned =  [ [] for x in range(t.topology.n_residues//BIN_SIZE) ]


    # TODO: May want to use np.random.choice to randomly select the binned lipids, though the choice of lipids to sample is already random
    # For each group...
    for g in range(n_bins):

        # Pick the slice of detections that are relevant to it
        _detections = [x for x in detections[spot_num][g*BIN_SIZE:BIN_SIZE*(g+1)]]


        avg_I = np.mean(_detections)
        tot_I = np.sum(_detections, axis=0)

        delta_I = [tot_I[x] - avg_I for x in range(len(tot_I))]


        binned_tots[spot_num][g] = tot_I
        binned_avgs[spot_num][g] = avg_I
        binned_dI[spot_num][g] = delta_I

    # binned_tot = np.sum(binned_tots, axis=0) - np.mean(binned_tots)
    binned_tot[spot_num] = np.mean(binned_dI[spot_num], axis=0)

In [742]:
np.shape(binned_dI)

## Plotting

### Intensity Traces
Plot the intensity traces for the individual lipids, and for the summed intensities.

In [743]:
################## Plot intensity traces #############################
# Plot the intensities of detections associated with each lipid
plt.subplot(221)
# for _data in binned_dI:
#     plt.plot(np.arange(0, len(t), 1), _data, marker='o', markersize=.5 ) 
plt.xlabel("Time (ns)")
plt.ylabel("Intensity")
plt.title("Intensity Trace")

# Plot sum of intensities (i.e. what a detector would see)
plt.subplot(222)
for spot_num in range(N_SPOTS):
    plt.plot(np.arange(0, len(t), 1), binned_tot[spot_num], marker='None', label="Spot %d" % spot_num)
# plt.plot(np.arange(0, len(t), 1), binned_tot[0], marker='None', markersize=.5 ) 
# summed_data = np.sum(intensities, axis=0)
# #plt.plot(np.arange(0,len(summed_data), 1), summed_data, marker = 'o', markersize=.1, )
# plt.plot(np.arange(0,len(binned), 1), [x[1] for x in binned], marker = 'o', markersize=.1, )
plt.legend()
plt.xlabel("Time (ns)")
plt.ylabel("Intensity")
plt.title("Summed Intensity Trace")

### Autocorrelations
Plot the autocorrelation functions for the individual lipids tracked, and for the summed intensities of all of them.

In [744]:
# Don't set the max lag to greater than the number of datapoints..
# max_lag = min([100 * int(tauD), (len(t) -1) * STEP])
max_lag = (len(t)-1) * STEP

tauD = FWHM**2 / (4*D)

# Individual data
plt.subplot(221)
plt.xscale('log')

# for _data in binned_dI:
#     plt.acorr(_data, maxlags=max_lag, usevlines=False, linestyle='-', marker="None", normed=True)

plt.xlabel("Time lag (ns)")
plt.title("Normalized Autocorrelation")

# Plot model curve
_x = np.arange(0,len(t), 1)
_G =(1 + _x / tauD)**(-1)
plt.plot(_x, _G, linestyle='--', linewidth=4)
    
# Plot tau_D, diffusion time
plt.axvline(tauD)
plt.xticks(list(plt.xticks()[0]) + [tauD], list(plt.xticks()[0]) + ['tau_D'])
plt.xlim([0,max_lag])


# Summed data
plt.subplot(222)
plt.xscale('log')

for spot_num in range(N_SPOTS):
    plt.acorr(binned_tot[spot_num], maxlags=max_lag, usevlines=False, linestyle='-', marker="None", normed=True, label="Data - Spot %d" % spot_num)


# Plot model curve
_x = np.arange(0,len(t), 1)
# _G = (1 + _x / tauD)**(-1)
plt.plot(_x, autocorr_model(_x, D*1), linestyle='--', linewidth=4, label="Model")

plt.xlabel("Time lag (ns)")
plt.title("Normalized Autocorrelation")
    
# Plot tau_D, diffusion time
plt.axvline(tauD)
plt.xticks(list(plt.xticks()[0]) + [tauD], list(plt.xticks()[0]) + ['tau_D'])
plt.xlim([0,max_lag])
plt.legend()

## Determine diffusion coefficient from FCS data

The following cells determine the diffusion coefficient from the FCS data using two different techniques. The binned and summed data are treated separately. First, a curve is fit to the autocorrelation curve using the diffusion constant as the parameter.

### Curve Fit Method

Using the equation for diffusion in a membrane presented by Schwille$^{[1]}$, attempt to fit the FCS data to a function of the form

$G(t) = \frac{1}{N} \left(1 + \frac{t}{\tau_D} \right)^{-1}$, where

$\tau_D = \frac{w_{xy}^2}{4 D}$

Since the autocorrelation curves are normalized, $N$ is set to 1.


#### Binned data

In [745]:
# _x = np.arange(0, len(t), 1)

# plt.xscale('log')
# optimals = []
# covariances = []
# i = 0
# for _data in binned_dI:
#     i+= 1
#     print(i, end="\r")
    
#     autocorrelated = autocorrelate(_data)
    
#     _optimal, _covariance = curve_fit(autocorr_model, _x, autocorrelated, p0=D)
#     optimals.append(_optimal)
#     covariances.append(_covariance)
    
#     plt.plot(_x, autocorrelated)
#     plt.plot(_x, autocorr_model(_x, _optimal), linestyle='--')
    
# var = [np.sqrt(np.diag(x)) for x in covariances]
    
# print("Average D from data was %f +- %f" % (np.mean(optimals), np.mean(var)))
# plt.xlim([0,len(t)])

#### Summed data

In [746]:
# autocorrelated = autocorrelate(binned_tot)

# normalized = binned_tot/np.linalg.norm(binned_tot)

# optimal, covariance = curve_fit(autocorr_model, _x, autocorrelated, p0=D)

# print("D was set at %f" % D)
# print("D estimated at %f +- %f." % (optimal, np.sqrt(np.diag(covariance))))

# plt.xscale('log')

# # Orange line is using the known diffusion constant
# plt.plot(_x, autocorr_model(_x, D), linestyle='--', label="Model - Known D")
# plt.acorr(binned_tot, maxlags=max_lag, usevlines=False, linestyle='-', marker="None", normed=True, label="Data")

# # Black line is using the calculated diffusion constant
# plt.plot(_x, autocorr_model(_x, optimal), linestyle='--', color='black', label="Model - Calced D")
# var = np.sqrt(np.diag(covariance))

# _y1 = autocorr_model(_x, optimal+var)
# _y2 = autocorr_model(_x, optimal-var)

# plt.fill_between(_x, _y1, _y2)

# plt.xlim([0,len(t)])
# plt.legend()

### .5 crossing method
This method determines $\tau_D$ first by looking for where the normalized autocorrelation crosses 0.5, then computes the diffusion coefficient from that using the same formula as above, solved for $D$

$D = \frac{w_{xy}^2}{4 \tau_D}$

#### Binned data

In [752]:
_x = np.arange(0, len(t), 1)
ax = plt.gca()

crossings = []

plt.xscale('log')
for spot_num in range(N_SPOTS):
    for _data in binned_dI[spot_num]:

        autocorrelated = autocorrelate(_data)

        interpolated = interp1d(_x, autocorrelated - .5)
        crossing = fsolve(interpolated, tauD*.2)
        crossings.append(crossing)

        
        color = ax._get_lines.get_next_color()
        
        plt.plot(_x, autocorrelated, color=color)

        calced_D = FWHM**2 / (4 * crossing)
        plt.plot(_x, autocorr_model(_x, calced_D), color=color, linestyle='--')
    

calced_D = [FWHM**2 / (4 * x) for x in crossings]


plt.plot(_x, autocorr_model(_x, D), color='k', linestyle='-', linewidth=2, label="Model - Expected")
    

# Plot tau_D, diffusion time
calc_tauD = FWHM**2 / (4 * np.mean(calced_D))
plt.axvline(calc_tauD)
plt.axvline(tauD)

plt.xticks(list(plt.xticks()[0]) + [calc_tauD, tauD], list(plt.xticks()[0]) + ['calculated tau_D', 'expected tau_D'])

plt.xlim([0,len(t)])

print("Average tau_D is %f +- %f" % (np.mean(crossings), np.std(crossings)))
print("Average diffusion constant is %.3f +- %.4f" % (np.mean(calced_D), np.std(calced_D)))

#### Summed data

In [753]:
# plt.subplot(211)
plt.xscale('log')
ax = plt.gca()
_x = np.arange(0, len(t), 1)

Ds = []

for spot_num in range(N_SPOTS):
    
    autocorrelated = autocorrelate(binned_tot[spot_num])

    interpolated = interp1d(_x, autocorrelated - .5)
    crossings = fsolve(interpolated, tauD*.5)
    
    print(crossings)

    calced_D = [FWHM**2 / (4 * _tauD) for _tauD in crossings]

    color = ax._get_lines.get_next_color()
    plt.plot(_x, autocorrelated, color=color, label="Data - Spot %d" % spot_num)
    plt.plot(_x, autocorr_model(_x, calced_D[0]), color=color, linestyle='--', label="Model - Spot %d" % spot_num)

    print("Calculated D is %.3f, tau_D is %.3f" % (np.mean(calced_D), FWHM**2 / (4 * calced_D[0])))
    
    Ds.append(calced_D)


plt.plot(_x, autocorr_model(_x, D), color='k', linestyle='-', linewidth=2, label="Model - Expected")
    
# Plot tau_D, diffusion time

calc_tauD = FWHM**2 / (4 * np.mean(Ds))
print("\n\nAvg. D is %.3f +- %.4f, yielding a tau_D of %.2f compared to the expected %.2f" % (np.mean(Ds), np.std(Ds), calc_tauD, tauD))

plt.axvline(calc_tauD)
plt.axvline(tauD)

plt.xticks(list(plt.xticks()[0]) + [calc_tauD, tauD], list(plt.xticks()[0]) + ['calculated tau_D', 'expected tau_D'])

plt.legend()

plt.xlim([0,len(t)])

# Bibliography

[1] Chiantia, Salvatore, Jonas Ries, and Petra Schwille. "Fluorescence correlation spectroscopy in membrane structure elucidation." Biochimica et Biophysica Acta (BBA)-Biomembranes 1788.1 (2009): 225-233.

[2] Zgorski, Andrew, and Edward Lyman. "Toward Hydrodynamics with Solvent Free Lipid Models: STRD Martini." Biophysical journal 111.12 (2016): 2689-2697.